In [11]:
import torch.nn as nn
import torch
import numpy as np

In [5]:
def cosine_scheduler(base_value, final_value, epochs, niter_per_ep, warmup_epochs=0, start_warmup_value=0):
    warmup_schedule = np.array([])
    warmup_iters = warmup_epochs * niter_per_ep
    if warmup_epochs > 0:
        warmup_schedule = np.linspace(start_warmup_value, base_value, warmup_iters)
    iters = np.arange(epochs * niter_per_ep - warmup_iters)
    schedule = final_value + 0.5 * (base_value - final_value) * (1 + np.cos(np.pi * iters / len(iters)))

    schedule = np.concatenate((warmup_schedule, schedule))
    assert len(schedule) == epochs * niter_per_ep
    return schedule

In [6]:
batch_size_per_replica = 64
num_gpus = 4
total_batch_size = batch_size_per_replica * num_gpus
data_size = 300000
warmup_epochs = 50
epochs = 200
base_value = 0.3*(total_batch_size/256)
final_value = 0.0048
niter_per_ep = data_size // total_batch_size
cosine_scheduler(base_value, final_value, epochs, niter_per_ep, warmup_epochs)

array([0.00000000e+00, 5.12391330e-06, 1.02478266e-05, ...,
       4.80000021e-03, 4.80000009e-03, 4.80000002e-03])

So the initial learning rate seems to be 0

## Load DINO ResNet50

In [17]:
import torchvision
# checkpoint_path = "dino_resnet50_pretrain_full_checkpoint.pth"
model = torchvision.models.__dict__["resnet50"]()
# embed_dim = model.fc.weight.shape[1]
# model.fc = nn.Identity()
# url = "dino_resnet50_pretrain/dino_resnet50_pretrain.pth"
# state_dict = torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/dino/" + url)
state_dict = torch.load(checkpoint_path)
model.load_state_dict(state_dict)

RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "conv1.weight", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "layer1.0.conv1.weight", "layer1.0.bn1.weight", "layer1.0.bn1.bias", "layer1.0.bn1.running_mean", "layer1.0.bn1.running_var", "layer1.0.conv2.weight", "layer1.0.bn2.weight", "layer1.0.bn2.bias", "layer1.0.bn2.running_mean", "layer1.0.bn2.running_var", "layer1.0.conv3.weight", "layer1.0.bn3.weight", "layer1.0.bn3.bias", "layer1.0.bn3.running_mean", "layer1.0.bn3.running_var", "layer1.0.downsample.0.weight", "layer1.0.downsample.1.weight", "layer1.0.downsample.1.bias", "layer1.0.downsample.1.running_mean", "layer1.0.downsample.1.running_var", "layer1.1.conv1.weight", "layer1.1.bn1.weight", "layer1.1.bn1.bias", "layer1.1.bn1.running_mean", "layer1.1.bn1.running_var", "layer1.1.conv2.weight", "layer1.1.bn2.weight", "layer1.1.bn2.bias", "layer1.1.bn2.running_mean", "layer1.1.bn2.running_var", "layer1.1.conv3.weight", "layer1

In [23]:
for module_key, module_value in state_dict["student"].items(): print(module_key, module_value.shape)

module.backbone.conv1.weight torch.Size([64, 3, 7, 7])
module.backbone.bn1.weight torch.Size([64])
module.backbone.bn1.bias torch.Size([64])
module.backbone.bn1.running_mean torch.Size([64])
module.backbone.bn1.running_var torch.Size([64])
module.backbone.bn1.num_batches_tracked torch.Size([])
module.backbone.layer1.0.conv1.weight torch.Size([64, 64, 1, 1])
module.backbone.layer1.0.bn1.weight torch.Size([64])
module.backbone.layer1.0.bn1.bias torch.Size([64])
module.backbone.layer1.0.bn1.running_mean torch.Size([64])
module.backbone.layer1.0.bn1.running_var torch.Size([64])
module.backbone.layer1.0.bn1.num_batches_tracked torch.Size([])
module.backbone.layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
module.backbone.layer1.0.bn2.weight torch.Size([64])
module.backbone.layer1.0.bn2.bias torch.Size([64])
module.backbone.layer1.0.bn2.running_mean torch.Size([64])
module.backbone.layer1.0.bn2.running_var torch.Size([64])
module.backbone.layer1.0.bn2.num_batches_tracked torch.Size([])
modul

In [ ]:
2048, 4096, 256